First Xgboost

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500)
sns.set(style="white", color_codes=True)

In [ ]:
train_house_df = pd.read_csv('../input/train.csv')
test_house_df = pd.read_csv('../input/test.csv')
macro_df = pd.read_csv('../input/macro.csv')

In [ ]:
macro_df.shape

In [ ]:
train_house_df.shape

In [ ]:
test_house_df.shape

In [ ]:
target = 'price'
IDcol = 'id'

In [ ]:
feature_df = train_house_df.drop(['price_doc'], axis=1 )

In [ ]:
merge_df = feature_df.append(test_house_df)

In [ ]:
imp_column=['full_sq' , 'life_sq' , 'floor' , 'max_floor' , 'num_room' , 'kitch_sq' , 'state' , 'build_year' , 'material']
merge_df['house_age'] = 2019  - merge_df['build_year']
imp_column.append('house_age')

imp_column

In [ ]:
from sklearn.preprocessing import Imputer

imp  =  Imputer(missing_values='NaN' ,strategy='median' , axis=1)

merge_df[imp_column] = imp.fit_transform(merge_df[imp_column])

In [ ]:
merge_df['log_full_sq'] =  np.log1p(merge_df['full_sq'])
merge_df['log_life_sq'] =  np.log1p(merge_df['life_sq'])

In [ ]:
merge_df['product_type'][merge_df['product_type'].isnull()]='Investment'

In [ ]:
main_merge_df  =  merge_df[['log_full_sq' , 'log_life_sq' , 'floor' , 'max_floor' , 'num_room' , 'kitch_sq' , 'state' , 'house_age', 'material','product_type' , 'sub_area']]

In [ ]:
from sklearn.preprocessing import LabelEncoder

def createDummy(df , var_mod):
    le = LabelEncoder()
    #var_mod = ['PROD_ABBR','STATE_ABBR' ]
    le = LabelEncoder()
    for i in var_mod:
        df[i] = le.fit_transform(df[i])

    #One Hot Coding:
    df = pd.get_dummies(df, columns=var_mod)
    return df

In [ ]:
other_column =  merge_df.columns.drop(imp_column)

In [ ]:
cat_columns  = merge_df[other_column].select_dtypes(exclude=['float64' , 'int64']).columns
num_columns  = merge_df[other_column].select_dtypes(include=['float64' , 'int64']).columns
macro_cat_columns  = macro_df.select_dtypes(exclude=['float64' , 'int64']).columns

macro_num_columns  = macro_df.select_dtypes(include=['float64' , 'int64']).columns


In [ ]:
from sklearn.preprocessing import Imputer

imp  =  Imputer(missing_values='NaN' ,strategy='median' , axis=1)

macro_df[macro_num_columns] = imp.fit_transform(macro_df[macro_num_columns])

In [ ]:
imp  =  Imputer(missing_values='NaN' ,strategy='median' , axis=1)

merge_df[num_columns] = imp.fit_transform(merge_df[num_columns])

In [ ]:
main_merge_df[cat_columns] = merge_df[cat_columns]
main_merge_df[num_columns] = merge_df[num_columns]

In [ ]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['price'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['price'].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['price'], dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
#Choose all predictors except target & IDcols
#predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, train_house_df, test_house_df)